<a href="https://colab.research.google.com/github/justinfmccarty/epwmorph/blob/main/Calculate%20Heat%2BCool%20Periods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Determining Heating and Cooling Seasons

This notebook uses effective tempreature (Houghton and Yaglou, 1923) to calculate a sliding effective temperture for each day of the year and then detemrine where to bracket out heating and cooling seasons following Wang et al (2015).

*Houghten, F., Yaglou, C., 1923. Determining lines of equal comfort. ASHVE Trans. 29, 163–176.*

*Wang Y, Wang L, Liu Y, Ding P, Deng S, Long E. Defining air-conditioning and heating seasons based on human thermal perception and building energy efficiency. Urban Clim 2015;14:544–53. doi:10.1016/j.uclim.2015.09.008.*

In [ ]:
!pip install ladybug-core
!pip install ladybug-pandas

In [1]:
import random
import numpy as np
import pandas as pd
import urllib
import zipfile
import ladybug_pandas as lbp
from ladybug.epw import EPW

In [14]:
def calcET(dbt,rh):
  effective_temp = dbt - 0.4*(1-rh)
  return effective_temp

def calcSET(effectivetemps):
  SETs = []
  for i in list(range(0,len(effectivetemps)-4)):
    SETdaily = (sum((effectivetemps[i],effectivetemps[i+1],effectivetemps[i+2],effectivetemps[i+3],effectivetemps[i+4])))/5
    SETs.append(SETdaily)
  return SETs

def createSET(dbt,rh):
  idx = pd.date_range(freq="h", start="2018-01-01", periods=365*24)
  data = pd.DataFrame(data={'dbt': dbt,
                            'rh': rh},
                      index=idx)
  data['eff_temp'] = data.apply(lambda x: calcET(x['dbt'],x['rh']), axis=1 )
  data_daily = data.resample('D').mean()

  efftemps = data_daily['eff_temp'].values.tolist()
  efftemps[365:369] = efftemps[0:4]

  SETs_list = calcSET(efftemps)
  return SETs_list

In [2]:
url = 'http://climate.onebuilding.org/WMO_Region_4_North_and_Central_America/CAN_Canada/BC_British_Columbia/CAN_BC_Vancouver.Intl.AP.718920_CWEC.zip'
urllib.request.urlretrieve(url,'/content/yvr.zip')

zf = zipfile.ZipFile('/content/yvr.zip', 'r')
zf.extractall('/content/yvr')
zf.close()

In [3]:
epw_path = '/content/yvr/CAN_BC_Vancouver.Intl.AP.718920_CWEC.epw'
epw = EPW(epw_path)
epw._import_data()
df = lbp.DataFrame.from_epw(epw)
df.head()

,Dry Bulb Temperature,Dew Point Temperature,Relative Humidity,Atmospheric Station Pressure,Extraterrestrial Horizontal Radiation,Extraterrestrial Direct Normal Radiation,Horizontal Infrared Radiation Intensity,Global Horizontal Radiation,Direct Normal Radiation,Diffuse Horizontal Radiation,Global Horizontal Illuminance,Direct Normal Illuminance,Diffuse Horizontal Illuminance,Zenith Luminance,Wind Direction,Wind Speed,Total Sky Cover,Opaque Sky Cover,Visibility,Ceiling Height,Present Weather Observation,Present Weather Codes,Precipitable Water,Aerosol Optical Depth,Snow Depth,Days Since Last Snowfall,Albedo,Liquid Precipitation Depth,Liquid Precipitation Quantity
2017-01-01 00:00:00,2.0,1.9,99.0,103210.0,0.0,9999.0,297.0,0.0,0.0,0.0,0.0,0.0,0.0,999900.0,45.0,1.7,10.0,10.0,0.2,0.0,0.0,999999099.0,999.0,0.999,0.0,88.0,0.0,0.0,0.0
2017-01-01 01:00:00,1.1,1.1,100.0,103340.0,0.0,9999.0,292.0,0.0,0.0,0.0,0.0,0.0,0.0,999900.0,0.0,0.0,10.0,10.0,0.2,60.0,0.0,999999099.0,999.0,0.999,0.0,88.0,0.0,0.0,0.0
2017-01-01 02:00:00,0.5,0.4,99.0,103420.0,0.0,9999.0,281.0,0.0,0.0,0.0,0.0,0.0,0.0,999900.0,0.0,0.0,9.0,9.0,0.2,3000.0,0.0,999999099.0,999.0,0.999,0.0,88.0,0.0,0.0,0.0
2017-01-01 03:00:00,0.2,0.0,99.0,103460.0,0.0,9999.0,274.0,0.0,0.0,0.0,0.0,0.0,0.0,999900.0,0.0,0.0,8.0,8.0,0.8,3000.0,0.0,999999099.0,999.0,0.999,0.0,88.0,0.0,0.0,0.0
2017-01-01 04:00:00,0.0,-0.2,98.0,103460.0,0.0,9999.0,259.0,0.0,0.0,0.0,0.0,0.0,0.0,999900.0,0.0,0.0,3.0,3.0,0.8,77770.0,0.0,999999099.0,999.0,0.999,0.0,88.0,0.0,0.0,0.0


In [4]:
# the ladybug pandas dataframe is not really a pandas dataframe
# had to extract the data line by line instead of as a series to list operation

dbt_list = []
rh_list = []

for i in df['Dry Bulb Temperature']:
  dbt_list.append(i)

for i in df['Relative Humidity']:
  rh_list.append(i/100)
  

In [ ]:
SETs = createSET(dbt_list,rh_list)
SETs = np.round(SETs,2)
SETs